In [1]:
import regex as re
import pandas as pd
from src.utils.preprocessing import *

c:\Users\nasta\miniconda3\envs\ada3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def find_difference(string1, string2):
    lines1 = [line.strip() for line in string1.split('\n')]
    lines2 = [line.strip() for line in string2.split('\n')]
    
    diff_lines = [line for line in lines1 if line not in lines2]
    diff_string = '\n'.join(diff_lines)
    
    return diff_string

In [3]:
valid_data = pd.read_csv('data/version1.4_ours/valid/discharge.csv.gz', compression='gzip')
gold_data = pd.read_csv('data/version1.4_ours/valid/discharge_target.csv.gz', compression='gzip')

In [4]:
valid_data = valid_data.head(100)

In [5]:
valid_data = build_combined_discharge(valid_data, gold_data)
valid_data['clean_discharge'] = valid_data[['text','brief_hospital_course', 'discharge_instructions']] \
    .progress_apply(axis = 1,func = lambda x: remove_bhc_di(x['text'], x['brief_hospital_course'], x['discharge_instructions'], 'bhc'))

100%|██████████| 100/100 [00:00<00:00, 5000.24it/s]


In [6]:
first_valid_gold = valid_data['clean_discharge']
#print(valid_data['clean_discharge'][0])

In [7]:
valid_data['reconstructed'] = ''

for i in range(len(valid_data)):
    first_valid = valid_data['text'][i]
    valid_data['reconstructed'][i] = extract_sex(first_valid) + extract_service(first_valid) + extract_allergies(first_valid) + extract_chief_complaint(first_valid) \
    + extract_major_surgical_procedures(first_valid) + extract_history_of_present_illness(first_valid) \
    + extract_past_medical_history(first_valid) + extract_social_history(first_valid) + extract_family_history(first_valid) \
    + extract_physical_exam(first_valid) + extract_pertinent_results(first_valid) + extract_medication_on_admission(first_valid) \
    + extract_discharge_medications(first_valid) + extract_discharge_disposition(first_valid) \
    + extract_discharge_diagnosis(first_valid) + extract_discharge_condition(first_valid)

reconstructed = valid_data['reconstructed']
#print(valid_data['reconstructed'][0])

C:\Users\nasta\AppData\Local\Temp\ipykernel_14416\234841972.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['reconstructed'][i] = extract_sex(first_valid) + extract_service(first_valid) + extract_allergies(first_valid) + extract_chief_complaint(first_valid) \
C:\Users\nasta\AppData\Local\Temp\ipykernel_14416\234841972.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data['reconstructed'][i] = extract_sex(first_valid) + extract_service(first_valid) + extract_allergies(first_valid) + extract_chief_complaint(first_valid) \
C:\Users\nasta\AppData\Local\Temp\ipykernel_14416\234841972.py:5: SettingWit

In [16]:
print(first_valid_gold[20])

 
Name:  ___               Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: SURGERY
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___
 
Chief Complaint:
single car ___
 
Major Surgical or Invasive Procedure:
___ Posterior Fusion Laminectomy C6-C7  
___ FUSION CERVICAL ANTERIOR WITH INSTRUMENTATION C6-C7 

 
History of Present Illness:
___ s/p transferred to ___ from ___, where she was 
originally brought in by EMS due to MVC. Patient was found 
unrestrained and lethargic in a single vehicle collision the 
morning of ___, found wedged between the dashboard and steering 
wheel; no report of prolonged extrication or ejection. VSS on 
EMS transfer, but upon presentation to ___, intubated in ED 
for combativeness, altered mental status, and airway protection. 
Initial CT scanning at ___ showed a CT head with SAH, and 
numerous injuries including superior facet fx at C7; widened 
facet at C

In [9]:
reconstructed[0]

'Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nChief Complaint:\nWorsening ABD distension and pain \n\nMajor Surgical or Invasive Procedure:\nParacentesis\n\nHistory of Present Illness:\n___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, \nbioplar, PTSD, presented from OSH ED with worsening abd \ndistension over past week.  \nPt reports self-discontinuing lasix and spirnolactone ___ weeks \nago, because she feels like "they don\'t do anything" and that \nshe "doesn\'t want to put more chemicals in her." She does not \nfollow Na-restricted diets. In the past week, she notes that she \nhas been having worsening abd distension and discomfort. She \ndenies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, \ndysuria. She had food poisoning a week ago from eating stale \ncake (n/v 20 min after food ingestion), which resolved the same \nday. She denies other recent illness or sick contacts. She notes \nthat she has been noticin

In [11]:
for i in range(100):#len(reconstructed)):
    print('----------------------------------------------------------')
    print('index:', i)
    difference = find_difference(first_valid_gold[i], reconstructed[i])
    print(difference)

----------------------------------------------------------
index: 0
Name:  ___                     Unit No:   ___
Admission Date:  ___              Discharge Date:   ___
Date of Birth:  ___             Sex:   F
Attending: ___
Followup Instructions:
----------------------------------------------------------
index: 1
Name:  ___               Unit No:   ___
Admission Date:  ___              Discharge Date:   ___
Date of Birth:  ___             Sex:   M
Attending: ___
Followup Instructions:
----------------------------------------------------------
index: 2
Name:  ___                 Unit No:   ___
Admission Date:  ___              Discharge Date:   ___
Date of Birth:  ___             Sex:   F
Attending: ___.
Followup Instructions:
----------------------------------------------------------
index: 3
Name:  ___                       Unit No:   ___
Admission Date:  ___              Discharge Date:   ___
Date of Birth:  ___             Sex:   M
Attending: ___.
Followup Instructions:
----------